In [ ]:
from datasets import load_dataset
from transformers import ASTForAudioClassification, AutoFeatureExtractor

In [11]:
ds = load_dataset("gilkeyio/AudioMNIST")

In [12]:
df = ds['train'].to_pandas()
df.head()

,speaker_id,audio,digit,gender,accent,age,native_speaker,origin
0,59,{'bytes': b'RIFF 3\x01\x00WAVEfmt \x10\x00\x00...,7,1,German,31.0,False,"Europe, Germany, Berlin"
1,59,{'bytes': b'RIFF\xb0<\x01\x00WAVEfmt \x10\x00\...,7,1,German,31.0,False,"Europe, Germany, Berlin"
2,59,{'bytes': b'RIFFZ\xce\x00\x00WAVEfmt \x10\x00\...,2,1,German,31.0,False,"Europe, Germany, Berlin"
3,59,{'bytes': b'RIFFV\xfb\x00\x00WAVEfmt \x10\x00\...,3,1,German,31.0,False,"Europe, Germany, Berlin"
4,59,{'bytes': b'RIFF\x02\x1a\x01\x00WAVEfmt \x10\x...,9,1,German,31.0,False,"Europe, Germany, Berlin"
